# Data and Model Parallelism in PyTorch

![Single GPU training data flow](images/single-gpu-training.png)

![Data and model parallelism approaches differ in how they take advantage of multiple GPUs](images/parallelisms.png)

| Operation | Estimated Time for 100 kB | Time (microseconds) | Relative to moving data to/from global memory |
| --- | --- | --- | --- |
| Simple scalar computation	 |  | 0.005	| 0.01 |
| Move data to global memory on the same GPU	|  0.75 µs	| 0.75	| 1.00 |
| Move data to another GPU on the same NVLINK domain | 2.5 µs	| 2.5 | 3.50 |
| Move data to CPU memory over PCIe	| 10 µs	| 10 |	13.0 |
| Move data to another GPU on another node over Infiniband	| 1 ms	| 1000	| 1,300 |
| Move data to a ocal SSD filesystem | 30 ms	| 33500	| 45,000 |
| Move data to a networked filesystem |	80 ms	| 85000	| 110,000 |